In [66]:
import pandas as pd
import numpy as np
import os 
import re
import ast
pd.options.display.max_columns = None

Cargaremos los dos ficheros de datos

In [67]:
df_limp = pd.read_csv("attacks_limpieza_5.csv", index_col=0)
df_limp.head()

,case_number,type,country,area,location,activity,name,sex_,age,time,species_,href,fatal_(y/n),injury,date,sex_mode,type_mode,year,age_new
0,1800.00.00,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, all onboard were killed by sharks",1800,F,Unprovoked,1927.133059,27.267683
1,1797.05.28.R,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,FATAL,Reported May-28-1797,M,Unprovoked,1927.133059,27.267683
2,1792.09.12,Provoked,england,NaN,On the Thames River,Fishing,male,M,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, PROVOKED INCIDENT",12-Sep-1792,M,Provoked,1927.133059,27.267683
3,1791.00.00,Unprovoked,australia,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, ""bitten in two""",1791,F,Unprovoked,1927.133059,27.267683
4,1788.05.10,Boat,australia,New South Wales,Sydney Harbor,Fishing,boat,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,N,"No injury to occupants, shark bit oar and rudder",10-May-1788,M,Boat,1927.133059,27.267683


In [86]:
clima = pd.read_csv("etl_1.csv", index_col = 0)
clima.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA
2,9,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",27,-1,11,1013,none,1,0,95,3,USA
3,12,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",26,-1,11,1012,none,1,0,100,3,USA
4,15,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",27,-1,11,1013,none,1,0,100,3,USA


In [69]:
clima.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'pais'],
      dtype='object')

Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer:
 - USA
 - Australia
 - New Zealand
 - South Africa
 - Papua New Guinea

In [70]:
filtrado_paises = df_limp[(df_limp['country'] == 'australia') 
                  |(df_limp['country'] == 'usa')|(df_limp['country'] == 'south africa') | (df_limp['country'] == 'papua new guinea')
                  | (df_limp['country'] == 'new zealand')]

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

 - Groupby de rh_profile y wind_profile

In [87]:
clima_copy = clima.copy()

In [72]:
clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
temp2m                int64
lifted_index          int64
rh2m                  int64
msl_pressure          int64
prec_type            object
prec_amount           int64
snow_depth            int64
wind10m.direction     int64
wind10m.speed         int64
pais                 object
dtype: object

In [88]:
clima_copy['wind_profile']= clima_copy['wind_profile'].apply(ast.literal_eval)

In [89]:
x = clima_copy['wind_profile'].apply(pd.Series)

In [90]:
x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 100, 'speed': 4}","{'layer': '900mb', 'direction': 90, 'speed': 3}","{'layer': '850mb', 'direction': 75, 'speed': 3}","{'layer': '800mb', 'direction': 55, 'speed': 3}","{'layer': '750mb', 'direction': 45, 'speed': 2}","{'layer': '700mb', 'direction': 325, 'speed': 2}","{'layer': '650mb', 'direction': 330, 'speed': 2}","{'layer': '600mb', 'direction': 325, 'speed': 3}","{'layer': '550mb', 'direction': 335, 'speed': 3}","{'layer': '500mb', 'direction': 355, 'speed': 3}","{'layer': '450mb', 'direction': 285, 'speed': 3}","{'layer': '400mb', 'direction': 270, 'speed': 4}","{'layer': '350mb', 'direction': 235, 'speed': 3}","{'layer': '300mb', 'direction': 255, 'speed': 3}","{'layer': '250mb', 'direction': 250, 'speed': 3}","{'layer': '200mb', 'direction': 215, 'speed': 4}"
1,"{'layer': '950mb', 'direction': 100, 'speed': 4}","{'layer': '900mb', 'direction': 90, 'speed': 4}","{'layer': '850mb', 'direction': 85, 'speed': 3}","{'layer': '800mb', 'direction': 75, 'speed': 3}","{'layer': '750mb', 'direction': 50, 'speed': 2}","{'layer': '700mb', 'direction': 355, 'speed': 2}","{'layer': '650mb', 'direction': 325, 'speed': 2}","{'layer': '600mb', 'direction': 320, 'speed': 2}","{'layer': '550mb', 'direction': 335, 'speed': 3}","{'layer': '500mb', 'direction': 345, 'speed': 3}","{'layer': '450mb', 'direction': 280, 'speed': 3}","{'layer': '400mb', 'direction': 270, 'speed': 4}","{'layer': '350mb', 'direction': 215, 'speed': 3}","{'layer': '300mb', 'direction': 280, 'speed': 3}","{'layer': '250mb', 'direction': 230, 'speed': 3}","{'layer': '200mb', 'direction': 260, 'speed': 3}"
2,"{'layer': '950mb', 'direction': 95, 'speed': 4}","{'layer': '900mb', 'direction': 90, 'speed': 4}","{'layer': '850mb', 'direction': 80, 'speed': 3}","{'layer': '800mb', 'direction': 75, 'speed': 3}","{'layer': '750mb', 'direction': 50, 'speed': 2}","{'layer': '700mb', 'direction': 325, 'speed': 2}","{'layer': '650mb', 'direction': 305, 'speed': 2}","{'layer': '600mb', 'direction': 320, 'speed': 2}","{'layer': '550mb', 'direction': 335, 'speed': 3}","{'layer': '500mb', 'direction': 325, 'speed': 3}","{'layer': '450mb', 'direction': 270, 'speed': 4}","{'layer': '400mb', 'direction': 265, 'speed': 3}","{'layer': '350mb', 'direction': 205, 'speed': 3}","{'layer': '300mb', 'direction': 305, 'speed': 3}","{'layer': '250mb', 'direction': 245, 'speed': 3}","{'layer': '200mb', 'direction': 285, 'speed': 3}"
3,"{'layer': '950mb', 'direction': 95, 'speed': 4}","{'layer': '900mb', 'direction': 95, 'speed': 4}","{'layer': '850mb', 'direction': 90, 'speed': 3}","{'layer': '800mb', 'direction': 75, 'speed': 3}","{'layer': '750mb', 'direction': 45, 'speed': 2}","{'layer': '700mb', 'direction': 325, 'speed': 2}","{'layer': '650mb', 'direction': 315, 'speed': 2}","{'layer': '600mb', 'direction': 325, 'speed': 2}","{'layer': '550mb', 'direction': 325, 'speed': 2}","{'layer': '500mb', 'direction': 295, 'speed': 3}","{'layer': '450mb', 'direction': 260, 'speed': 5}","{'layer': '400mb', 'direction': 265, 'speed': 3}","{'layer': '350mb', 'direction': 210, 'speed': 3}","{'layer': '300mb', 'direction': 315, 'speed': 3}","{'layer': '250mb', 'direction': 260, 'speed': 3}","{'layer': '200mb', 'direction': 305, 'speed': 3}"
4,"{'layer': '950mb', 'direction': 90, 'speed': 4}","{'layer': '900mb', 'direction': 90, 'speed': 3}","{'layer': '850mb', 'direction': 95, 'speed': 3}","{'layer': '800mb', 'direction': 70, 'speed': 3}","{'layer': '750mb', 'direction': 45, 'speed': 2}","{'layer': '700mb', 'direction': 305, 'speed': 2}","{'layer': '650mb', 'direction': 310, 'speed': 2}","{'layer': '600mb', 'direction': 315, 'speed': 2}","{'layer': '550mb', 'direction': 305, 'speed': 2}","{'layer': '500mb', 'direction': 290, 'speed': 3}","{'layer': '450mb', 'direction': 255, 'speed': 4}","{'layer': '400mb', 'direction': 260, 'speed': 3}","{'layer': '350mb', 'direction': 200, 'speed': 2}","{'layer': '300mb', 'direction': 325, 'speed': 

In [93]:
clima_copy['rh_profile']= clima_copy['rh_profile'].apply(ast.literal_eval)

In [94]:
y = clima_copy['rh_profile'].apply(pd.Series) 

In [95]:
y

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 13}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -2}"
1,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -1}"
2,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 15}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 3}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -2}","{'layer': '300mb', 'rh': -2}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -1}"
3,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 15}","{'layer': '850mb', 'rh': 12}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': -1}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -2}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': 0}"
4,"{'layer': '950mb', 'rh': 11}","{'layer': '900mb', 'rh': 13}","{'layer': '850mb', 'rh': 15}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': -1}","{'layer': '700mb', 'rh': -3}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -3}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -2}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': 0}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"{'layer': '950mb', 'rh': 15}","{'layer': '900mb', 'rh': 15}","{'layer': '850mb', 'rh': 15}","{'layer': '800mb', 'rh': 15}","{'layer': '750mb', 'rh': 15}","{'layer': '700mb', 'rh': 15}","{'layer': '650mb', 'rh': 11}","{'layer': '600mb', 'rh': 10}","{'layer': '550mb', 'rh': 2}","{'layer': '500mb', 'rh': 6}","{'layer': '450mb', 'rh': 10}","{'layer': '400mb', 'rh': 7}","{'layer': '350mb', 'rh': 12}","{'layer': '300mb', 'rh': 8}","{'layer': '250mb', 'rh': -1}","{'layer': '200mb', 'rh': -4}"
60,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 7}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -1}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': 2}","{'layer': '500mb', 'rh': 1}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -3}","{'layer': '350mb', 'rh': -2}","{'layer': '300mb', 'rh': -1}","{'layer': '250mb', 'rh': -2}","{'layer': '200mb', 'rh': -4}"
61,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 16}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 3}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -2}","{'layer': '500mb', 'rh': -2}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': 2}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', '

In [96]:
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    clima_copy.insert(i, nombre, valores)

KeyError: 'rh'

In [ ]:
# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. 
# En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas 
# nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. 
# Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario 
# (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. 
# Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. 
# En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. 
# Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 